### 3.0 preload raw byte string content from pickle
+ raw contains 45 meeting minutes from San Jose city concoil.

In [1]:
import pickle

def load_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [2]:
raw      = load_obj('./council_minutes_dict.pkl')
articles = []
for (key,article) in raw.items():
    #article = byte_article.decode('ISO8859-1')
    articles.append(article)

### 3.1 search and separate content as per formal text pattern
+ identify and separate individual main topic and the content between.
+ identify sub topic within the content block.

+ uses regex as main usage.

In [3]:
# import regex and identify patterns for each content of interests.
import regex as re

# room for improvement. host defined pattern to backend/user defined pattern
# may subject to change between different pdf contents
patterns = {'maintitle':re.compile(r'\n{1}\s?(\d{0,2}[.]\s*.*?)\n'),
           'subtitle':re.compile(r'\s?(\d?[.]\d?\s*.*)'),
           'dollar':re.compile('([$]\d+([,.]\d+)*)'),
           'item':re.compile(r'\s(\d+-\d+\s+.*)\n'),
           'section':re.compile(r'\s((Sections?)\s\d+[.]\d+[.]\d+)'),
           'date':re.compile(r'([ADFJMNOS]\w* \d{1,2},\s?20\d+)')}

+ identifing recoganizable key words for main title recognition

In [4]:
# 

mainTopicLS =['CEREMONIAL',
              'CEREMONIALS',
              'ITEMS',
              'CONSENT',
              'CALENDAR',
              'STRATEGIC',
              'SUPPORT',
              'COMMUNITY',
             'ECONOMIC',
             'DEVELOPMENT',
             'NEIGHBORHOOD',
             'SERVICES',
             'TRANSPORTATION',
             'AVIATION',
             'ENVIRONMENTAL',
             'UTILITY',
             'PUBLIC',
             'SAFETY',
             'REDEVELOPMENT',
             'SUCCESSOR',
             'AGENCY',
             'LAND',
             'USE']
# The line below removes duplicates
mainTopicLS = list(dict.fromkeys(mainTopicLS)) 

+ main method to turn a string into a dataframe with main and subtitle marked.

In [5]:
import pandas as pd
import math

def qcRework (BLOCKS,date,pickLS = [],articleID = 1):
    Blocks = [[b,0,0,articleID,date] for b in BLOCKS]
    Blocks_result = []
    i = 0
    j = 0

    for b in Blocks:
        word_ls = b[0].split(' ')
        word_ls = [w for w in word_ls if w!=' ']
        word_ls = [w for w in word_ls if w!='']
        word_ls = [w for w in word_ls if w!='\n']
        try:
            # check the first item in the list and see if it is decimal number
            firstN = float(word_ls[0])
            subID = round((firstN-i)*100)
            # process, and realign the number
            if subID % 10 == 0 and j % 10 != 9:
                subID = round(subID/10)
            else:
                pass
            
            if math.floor(firstN)==i and subID == j+1:
                j = subID
            # print(word_ls[0],subID,j)
        except:
            pass
        
        if len(word_ls)<2:
            b[1] = j
            b[2] = i
            Blocks_result.append(b)
            continue
        
        if word_ls[-1] == word_ls[-1].upper() and word_ls[1] in pickLS:
            i = int(re.match('^\d+',word_ls[0]).group(0))
            j = 0
            b[2] = i
            Blocks_result.append(b)
            
        else:
            b[1] = j
            b[2] = i
            Blocks_result.append(b)
            
    df_blocks = pd.DataFrame(Blocks_result,columns = ['content','subID','mainID','artID','date'])
    return df_blocks

+ process all articles in the string list.

In [6]:
articles_in_blocks = []
i = 1

for a in articles:
    #print(i)
    BLOCKS = re.split(patterns['maintitle'],a)
    articles_in_blocks.append(qcRework(BLOCKS,re.search(patterns['date'],a).group(0),mainTopicLS,i))
    i+=1

+ visual validation

### 3.2 pack up all the subtitle content into one string.

+ function to identify the all the subtitle content pack into one

In [7]:
def df_pack(df_messy):
    df_compact = pd.DataFrame(columns = df_messy.columns)
    lastrow = None

    for id,row in df_messy.iterrows():
        if lastrow is None:
            lastrow = row
            continue
        if (row['subID'] == lastrow['subID']) and row['mainID'] == lastrow['mainID'] and row['artID'] == lastrow['artID']:
            lastrow['content'] += row['content']
        else:
            df_compact = df_compact.append(lastrow,ignore_index = True)
            lastrow = row
                  
    df_compact = df_compact.append(lastrow,ignore_index = True)
    
    return df_compact

+ process all the articles block list into dataframes.

In [8]:
df_articles = []

for a in articles_in_blocks:
    df_articles.append(df_pack(a))

### 3.3 add logic columns for specific contents

+ findPattern: find all the patterns in a string and return a logic value
+ addColumn: add a logic column for a specific pattern recognition by using findPattern function

In [9]:
def findPattern(s,pattern):
    sub = re.findall(pattern,s)
    #rint(sub)
    return sub !=[]

def addColumn(df,pattern,colname = None):
    if colname is None:
        colname = 'col' + str(len(df.columns))
    
    df[colname] = df['content'].apply(findPattern,args=[pattern])
    return df      


+ add hasDollar, hasItem and hasSection for all the artical dataframe

In [10]:
for df in df_articles:
    df = addColumn(df,patterns['dollar'],'hasDollar')
    df = addColumn(df,patterns['item'],'hasItem')
    df = addColumn(df,patterns['section'],'hasSection')


+ visual validation.

### 3.4 join all dataset in one dataframe

In [11]:
df_minutes =pd.DataFrame(columns= df_articles[0].columns)
df_minutes = df_minutes.append(df_articles,ignore_index = True)
df_minutes.head()

,content,subID,mainID,artID,date,hasDollar,hasItem,hasSection
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,0,1,"January 7,2020",False,True,False
1,1. CEREMONIAL ITEMSNone provided.\n\nCity of S...,0,1,1,"January 7,2020",False,False,False
2,2. CONSENT CALENDAR\nUpon motion by Councilmem...,0,2,1,"January 7,2020",False,False,False
3,2.1 Approval of City Council Minutes.None prov...,1,2,1,"January 7,2020",False,False,False
4,2.2 Final Adoption of Ordinances.20-056 Final ...,2,2,1,"January 7,2020",False,False,True


### 3.5 save and read
+ to and from feather file
+ need pyarrow installed.

In [12]:
import time

filename = 'city_SanJose_Minutes'

start = time.time()
df_minutes.to_feather(filename+'.feather')
df_read = pd.read_feather(filename+'.feather')
print(time.time()-start)

df_read.head()

0.15399813652038574


,content,subID,mainID,artID,date,hasDollar,hasItem,hasSection
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,0,1,"January 7,2020",False,True,False
1,1. CEREMONIAL ITEMSNone provided.\n\nCity of S...,0,1,1,"January 7,2020",False,False,False
2,2. CONSENT CALENDAR\nUpon motion by Councilmem...,0,2,1,"January 7,2020",False,False,False
3,2.1 Approval of City Council Minutes.None prov...,1,2,1,"January 7,2020",False,False,False
4,2.2 Final Adoption of Ordinances.20-056 Final ...,2,2,1,"January 7,2020",False,False,True


+ to and from csv file

In [13]:
start = time.time()
df_minutes.to_csv(filename + '.csv')
df_read = pd.read_csv(filename + '.csv')
print(time.time()-start)

df_read.head()

0.14599919319152832


,Unnamed: 0,content,subID,mainID,artID,date,hasDollar,hasItem,hasSection
0,0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,0,1,"January 7,2020",False,True,False
1,1,1. CEREMONIAL ITEMSNone provided.\n\nCity of S...,0,1,1,"January 7,2020",False,False,False
2,2,2. CONSENT CALENDAR\nUpon motion by Councilmem...,0,2,1,"January 7,2020",False,False,False
3,3,2.1 Approval of City Council Minutes.None prov...,1,2,1,"January 7,2020",False,False,False
4,4,2.2 Final Adoption of Ordinances.20-056 Final ...,2,2,1,"January 7,2020",False,False,True


### 3.6 save the dataframe into a sql database.

+ connect to meetingSticker DB and initiate cursor.

In [14]:
import sqlite3

con = sqlite3.connect('meetingSticker.db')
cur = con.cursor()


+ create table Minutes
+ submit changes

cur.execute('CREATE TABLE MINUTES(ID,content,subID,mainID,artID,hasDollar,hasItem,hasSection)')

con.commit()

+ overide MINUTES table with df_minutes in sql table.

In [15]:
df_minutes.to_sql('MINUTES',con,if_exists='replace',index=False)

+ operation validation

In [16]:
cur.execute('SELECT * FROM MINUTES')

for row in cur.fetchall():
    #print(row)
    pass

+ close database connection

In [17]:
con.close()